# 先导包

In [36]:
#先导包
import pandas as pd
import numpy as np
from efficient_apriori import apriori

## 数据读取分析

In [37]:
#先把数据读取进来
dataset=pd.read_csv("Market_Basket_Optimisation.csv",header=None)#header=None表示这个数据表没有表头，设置为空,否则会把第一行的数据当成表头
dataset#可以看到数据有很多空值，一共有20列 表示一个人最多买东西的时候 买了20种东西

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,butter,light mayo,fresh bread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,burgers,frozen vegetables,eggs,french fries,magazines,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7499,escalope,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 通过表中购买的商品名称 来构建transactions  准备构建好了放到算法里面

In [38]:
#想办法把数据存到 trasactions中
transactions=[]
#遍历一下dataframe中的数据 
for i in range(0,dataset.shape[0]):#遍历所有的行
    temp=[]
    for j in range(0,dataset.shape[1]): #在每一行中去遍历每一个列元素，dataset.shape[0]=7501 dataset.shape[1]=20
        #把每一个元素转成一个字符串str 并添加到temp列表中
        temp.append(str(dataset.values[i,j]))#这里range的范围分别取到7500和19 而不是7501和20 表格中最大的行列索引也刚好是7500与19
    #这个temp会保存每一行的购物篮的商品的名称的str类型 不过进行下一个for循环会被重新赋值为0 重新存
    if i<6:
        print(temp)    #跟预期的效果 是一样的 不过有太多的空值   解决方法见下一个cell里面的代码
        #print前5行吧 否则上传到github之后 全部显示出来 导致网页太长 

['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil']
['burgers', 'meatballs', 'eggs', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']
['chutney', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']
['turkey', 'avocado', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']
['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']
['low fat yogurt', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'na

In [39]:
#空值太多  可以先判断一下 是否是空值 然后再往里面添加
#遍历一下dataframe中的数据 
for i in range(0,dataset.shape[0]):#遍历所有的行
    temp=[]
    for j in range(0,dataset.shape[1]): #在每一行中去遍历每一个列元素，dataset.shape[0]=7501 dataset.shape[1]=20
        #把每一个元素转成一个字符串str 并添加到temp列表中
        #如果这个字符串是nan那就不添加了  
        if str(dataset.values[i][j])!='nan':
            temp.append(str(dataset.values[i,j]))#这里range的范围分别取到7500和19 而不是7501和20 表格中最大的行列索引也刚好是7500与19
    #这个temp会保存每一行的购物篮的商品的名称的str类型 不过进行下一个for循环会被重新赋值为0 重新存
    #然后每一次循环中的temp放到外面的大的trasactions[]中
    transactions.append(temp)
print(transactions[0:10])#这个时候就是一个大的列表trasactions中 放了很多个小的列表temp temp中的元素是购物的商品的字符串形式    
#print前10个吧 否则上传到github之后 全部显示出来 导致网页太长 

[['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil'], ['burgers', 'meatballs', 'eggs'], ['chutney'], ['turkey', 'avocado'], ['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea'], ['low fat yogurt'], ['whole wheat pasta', 'french fries'], ['soup', 'light cream', 'shallot'], ['frozen vegetables', 'spaghetti', 'green tea'], ['french fries']]


## transactions构建好了 来用apriori挖掘规则

In [40]:
#然后把这个trasactions放入到apriori中去 去跑一下频繁项集和关联规则
#包已经在最上面导入了 
# 挖掘频繁项集和频繁规则
itemsets, rules = apriori(transactions, min_support=0.02,  min_confidence=0.3)
print('频繁项集：\n', itemsets)#大的字典只有两个 12  大于最小支持度为0.02就是这些商品组合   没有三个商品组合还支持度大于0.02的
print('关联规则：\n', rules)#在最小支持度为0.02和最小置信度大于0.3的就是这些组合  {}->{}前面括号对括号的置信度

频繁项集：
 {1: {('olive oil',): 494, ('frozen smoothie',): 475, ('tomato juice',): 228, ('vegetables mix',): 193, ('low fat yogurt',): 574, ('honey',): 356, ('salmon',): 319, ('cottage cheese',): 239, ('avocado',): 250, ('almonds',): 153, ('shrimp',): 536, ('mineral water',): 1788, ('energy drink',): 200, ('green tea',): 991, ('burgers',): 654, ('meatballs',): 157, ('eggs',): 1348, ('turkey',): 469, ('milk',): 972, ('whole wheat rice',): 439, ('energy bar',): 203, ('whole wheat pasta',): 221, ('french fries',): 1282, ('soup',): 379, ('spaghetti',): 1306, ('frozen vegetables',): 715, ('cookies',): 603, ('cooking oil',): 383, ('champagne',): 351, ('chocolate',): 1229, ('chicken',): 450, ('oil',): 173, ('fresh tuna',): 167, ('tomatoes',): 513, ('red wine',): 211, ('pepper',): 199, ('ham',): 199, ('pancakes',): 713, ('grated cheese',): 393, ('fresh bread',): 323, ('escalope',): 595, ('ground beef',): 737, ('herb & pepper',): 371, ('strawberries',): 160, ('cake',): 608, ('hot dogs',): 243, ('br